In [ ]:
import pandas as pd
import folium

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [ ]:
import geopandas as gpd
from matplotlib import pyplot as plt

In [ ]:
points = pd.read_csv('points.csv', index_col=0)

In [ ]:
data = pd.read_csv('open_stp_data.csv')

In [ ]:
data = data[['addr_street', 'addr_number', 'addr_building', 'addr_letter', 'addr_district']]

In [ ]:
municipalities = gpd.read_file("ignore/municip_okrug/municip_okrug.shp")
municipalities = municipalities.dropna(subset=('geometry',))

In [ ]:
data = data.join(points, how='inner')

In [ ]:
data['geometry'] = data.apply(lambda x: Point(x.lng, x.lat), axis=1)
data = GeoDataFrame(data, crs={'init': 'epsg:4326'}, geometry='geometry')

In [ ]:
data = data.to_crs(municipalities.crs)

In [ ]:
municipalities = municipalities[['NAME', 'type', 'geometry']]

In [ ]:
type(municipalities)

In [ ]:
full = gpd.sjoin(municipalities, data, op='contains')

In [ ]:
full[(full['NAME'] == 'Пушкин') & (full['addr_district'] != 'Пушкинский')].tail(50).sort_values(by='index_right')

In [ ]:
full[(full['NAME'] == 'Сестрорецк') & (full['addr_district'] != 'Курортный')].tail(50).sort_values(by='index_right')

In [ ]:
full[(full['NAME'] == 'Зеленогорск') & (full['addr_district'] != 'Курортный')].tail(50).sort_values(by='index_right')

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
df.sort_values(by='comm_room_sum', ascending=False).head(10)

In [ ]:
df = pd.read_csv('data_with_coords.csv')
df.head(5)

#### Коммунальное жилье (плотность??? как правильно назвать то, что на хитмапе)

Для каждого дома найдем суммарное количество коммунальных КОМНАТ. Представим в виде heatmap.

In [ ]:
fnct = lambda x: sum([int(xx) for xx in x.split(', ')])
df['comm_room_sum'] = df['comm_room_num'].fillna(0).astype('str').map(fnct)
df['comm_intensity'] = df['comm_room_sum']/df['comm_room_sum'].max()
df['comm_room_sum'].quantile(q=0.95)

In [ ]:
m = folium.Map(location=[59.9393773,30.3181802], zoom_start=10)
HeatMap(data=df.dropna(subset=('comm_num',))[['lat', 'lng', 'comm_room_sum']].values.tolist(),
        max_val=70.0,
        radius=8, max_zoom=13).add_to(m)
m

#### Годы постройки

Сохранившиеся жилые многоквартирные здания дореволюционной постройки

In [ ]:
data_columns = ['data_buildingdate',]
num_df = (df.drop(data_columns, axis=1).join(df[data_columns].apply(pd.to_numeric, errors='coerce')))

In [ ]:
num_df = num_df.dropna(subset=('data_buildingdate',))

In [ ]:
m1 = folium.Map(location=[59.9393773,30.3181802], zoom_start=10)
HeatMap(data=num_df[num_df['data_buildingdate'] < 1917][['lat', 'lng']].values.tolist(),
        radius=8, max_zoom=13).add_to(m1)
m1

In [ ]:
df = df.dropna(subset=('lng', 'lat'))

In [ ]:
df['geometry'] = df.apply(lambda x: Point(x.lng, x.lat), axis=1)

In [ ]:
points = GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry='geometry')

In [ ]:
points

In [ ]:
municipalities = gpd.read_file("ignore/municip_okrug/municip_okrug.shp")
municipalities = municipalities.dropna(subset=('geometry',))

In [ ]:
municipalities.crs

In [ ]:
municipalities.plot(figsize=(10, 10))

In [ ]:
points = points.to_crs(municipalities.crs)

In [ ]:
municip_w_data = gpd.sjoin(municipalities, points, op='contains')

In [ ]:
municip_w_data

In [ ]:
municip_w_data.info()

In [ ]:
test = municip_w_data.sort_values(by='comm_room_sum', ascending=False).head(5)

In [ ]:
test['comm_room_num']

In [ ]:
type(municip_w_data)

In [ ]:
df.head()

In [ ]:
municip_w_data.head(2)

In [ ]:
# Groupby on unique area code & size
municip_w_data = municip_w_data.groupby('OBJECTID')
municip_w_data

In [ ]:
municip_w_data[municip_w_data['NAME'] == 'Пушкин']

In [ ]:
comm_rooms = municip_w_data.sum()['comm_room_sum']

In [ ]:
living_area = municip_w_data.sum()['data_livingarea']

In [ ]:
living_area = living_area.to_frame().reset_index()
living_area.columns = ['OBJECTID', 'living_area']

In [ ]:
comm_rooms = comm_rooms.to_frame().reset_index()
comm_rooms.columns = ['OBJECTID', 'comm_rooms']

In [ ]:
merged_districts.area

In [ ]:
municipalities

In [ ]:
comm_density.sort_values(by='comm_rooms', ascending=False)

In [ ]:
comm_density = municipalities[['OBJECTID', 'NAME', 'geometry']].merge(comm_rooms, on='OBJECTID', how='outer')
comm_density = comm_density.merge(living_area, on='OBJECTID', how='outer')
comm_density['density'] = comm_density['comm_rooms'] / comm_density['living_area']

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
comm_density.plot(column='density', cmap='YlGnBu', legend=True, ax=ax, edgecolor='k');

In [ ]:
comm_density['a_density'] = comm_density['comm_rooms'] / comm_density.area
fig, ax = plt.subplots(figsize=(15,15))
comm_density.plot(column='a_density', cmap='YlGnBu', legend=True, ax=ax, edgecolor='k');

In [ ]:
comm_density['coords'] = comm_density['geometry'].apply(lambda x: x.representative_point().coords[:])

In [ ]:
comm_density.head(5)

In [ ]:
comm_density['coords'] = [coords[0] for coords in comm_density['coords']]

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
comm_density.plot(column='comm_rooms', cmap='YlGnBu', legend=True, ax=ax, edgecolor='k', )
for idx, row in comm_density.iterrows():
    plt.annotate(s=row['NAME'], xy=row['coords'], horizontalalignment='center')

In [ ]:
comm_density['coords'] = comm_density['geometry'].apply(lambda x: x.representative_point().coords[:])